In [ ]:
# Import Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# Connect to database
engine = create_engine("sqlite:///politics_db.db", connect_args={'check_same_thread': False}, echo=True)

# Reflect database and tables into new models
Base = automap_base()

In [ ]:
Base.prepare(engine, reflect=True)
Candidates = Base.classes.candidates
Twitter = Base.classes.twitter

# Create our session from Python to the DB
session = Session(bind=engine)

In [ ]:
username = "@JoeBiden"

In [ ]:
# Get sentiment data 
results = session.query(Twitter.tweet_date, Twitter.sentiment, func.count(Twitter.id).label("tweet_count"))\
    .group_by(Twitter.tweet_date)\
    .group_by(Twitter.sentiment)\
    .filter(Twitter.username == username)\
    .all()

In [ ]:
# Convert query result into list of dictionaries
tweet_list = []

for result in results:
    if result.sentiment:
        sentiment = "Positive"
    else:
        sentiment = "Negative"
    tweet_dict = {"date": result.tweet_date, "sentiment": sentiment, "tweet_count": result.tweet_count}
    tweet_list.append(tweet_dict)